In [ ]:
from wavelet_denoise import WaveletDenoiser
from vibration_loader import VibrationDataLoader  # 你自己的主文件

loader = VibrationDataLoader.from_txt("your_file_path.txt")
denoiser = WaveletDenoiser(wavelet='db4', level=2)
denoised_data = denoiser.denoise_dataframe(loader.data, columns=["Channel1", "Channel2"])
loader.data = denoised_data  # 可替换原始数据或另存新变量

In [ ]:
# 读取npz
import numpy as np
data = np.load('output/legacy/my_processed_data_use_all_points.npz')
print(data.files)

In [ ]:
import os
from utils.dataprocess.vibration_data_loader import VibrationDataLoader  
from utils.dataprocess.wavelet_denoise import WaveletDenoiser
from utils.dataprocess.start_idx_visualized_select import StartIdxVisualizedSelect
from utils.dataprocess.debiasing import debias_csv_folder
from utils.visualize.data_processor import DataProcessor
from utils.visualize.visualization_generator import VisualizationGenerator
from loguru import logger
import sys
from utils.visualize.extract_timepoint import extract_timepoint_data, visualize_grid,export_filename_grid
import pandas as pd
import matplotlib
# Set up the matplotlib backend explicitly
# matplotlib.use('TkAgg')  # Use TkAgg backend which has good button support
%matplotlib inline
import matplotlib.pyplot as plt
# 配置日志记录
logger.configure(handlers=[{"sink": sys.stdout, "level": "INFO"},
                           {"sink": "logs/log.log", "level": "INFO"}])

# 确保输出目录存在
os.makedirs('output', exist_ok=True)
os.makedirs('logs', exist_ok=True)
os.makedirs('output/data_csv', exist_ok=True)
os.makedirs('output/data_csv_start-idx-reselected', exist_ok=True)
os.makedirs('output/data_csv_start-idx-reselected_debiased', exist_ok=True)


# from wavelet_denoise import WaveletDenoiser
import pandas as pd

df = pd.read_csv("output/data_csv/54.csv")
# 只保留低频节点 'a'（一级），或者更深层次的如 'aa', 'aaa'（更平滑趋势）
denoiser = WaveletDenoiser(wavelet='db6', level=6, keep_nodes=['aaaaaa'])  
denoised_df = denoiser.denoise_dataframe(df, columns=["Time Signal"])
fs = 1/df["Time"].diff().mean()  # 采样频率，单位 Hz
logger.info(f"采样频率是: {fs} Hz")
# 可视化对比
denoiser.plot_denoise_comparison(df, denoised_df, column="Time Signal", time_column="Time")
denoiser.plot_denoise_overlay(df, denoised_df, column="Time Signal", time_column="Time")
selected = denoiser.auto_select_trend_nodes(
    signal=df["Time Signal"].values,
    threshold_ratio=0.05,
    fs=fs,            # 采样频率，单位 Hz
    plot="band"         # 展示频带分布图，可选 "bar" 或 None
)
logger.info(f"自动选择的趋势节点: {selected}")
# 假设你对 level = 6，想看第 0 个路径（从 0 开始计）
freq_range = denoiser.get_node_frequency_range(node_index=0, fs=fs)
logger.info(f"路径 0 的频率范围是: {freq_range[0]:.2f} Hz 到 {freq_range[1]:.2f} Hz")


In [ ]:
import pywt
import pandas as pd
wp = pywt.WaveletPacket(data=df["Time Signal"].values, wavelet='db6', maxlevel=9)
print([node.path for node in wp.get_level(9, 'freq')])